# All

## Setup

### Configuration Class

In [ ]:
class Config:
    FRAME_FOLDER = "Video_Frames"
    FACE_FOLDER = "Faces"
    MODEL_NAME = "VGG-Face"
    DISTANCE_METRIC = "euclidean"
    DETECTOR_BACKEND = "retinaface"
    DIRECTORY_FOLDER = os.getcwd()
    VIDEO_NAME = "Clip.mp4"
    UPLOADS_FOLDER = "Uploads"
    PKL_FOLDER = "PKL"
    INTERVAL = float(0.1)

# Create folders if they don't exist
FOLDERS = [
    Config.FRAME_FOLDER,
    Config.FACE_FOLDER,
    Config.PKL_FOLDER,
    Config.UPLOADS_FOLDER]

for folder in FOLDERS:
    if not os.path.exists(folder):
        os.makedirs(folder)

### Imports

In [1]:
!pip install deepface
!pip install fastapi
!pip install colabcode
!pip install pyngrok
!pip install retinaface
!pip install python-multipart
!pip install starlette

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 6.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=1bd2e56cc75f64c8b8e44a282b85f05a4b1814d35ad420a802a914080c72d5b9
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 815.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:0

In [2]:
import os
import shutil
import concurrent.futures
import cv2
from deepface import DeepFace
from fastapi import FastAPI, UploadFile
from fastapi.responses import HTMLResponse, JSONResponse, RedirectResponse
from retinaface import RetinaFace
from starlette.requests import Request
import time
import pickle
from os import path
from colabcode import ColabCode
from pyngrok import ngrok

# 3rd party dependencies
import pandas as pd
import cv2

# package dependencies
from deepface.commons import functions, distance as dst

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [3]:
DeepFace.build_model("VGG-Face")

vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:10<00:00, 53.2MB/s]


### API Setup

In [4]:
app = FastAPI()

In [5]:
cc = ColabCode(port=12000, code=False)

In [6]:
# Provide your Ngrok auth token
auth_token = "2VQ3N6dnV7bMo5PhJskNw6KOynM_43X2AJPHbCN42wdjt29jb"

# Set the auth token for Ngrok
ngrok.set_auth_token(auth_token)

### Find Function

In [7]:
def find(img_path, db_path, model_name="VGG-Face", distance_metric="euclidean", enforce_detection=True, detector_backend="retinaface", align=True, normalization="base", silent=False, bid = 0000):

    tic = time.time()

    # -------------------------------
    if os.path.isdir(db_path) is not True:
        raise ValueError("Passed db_path does not exist!")

    target_size = functions.find_target_size(model_name=model_name)

    # ---------------------------------------

    file_name = f"{bid}_representations_{model_name}.pkl"
    file_name = file_name.replace("-", "_").lower()

    if path.exists(db_path + "/" + file_name):

        if not silent:
            print(
                f"WARNING: Representations for images in {db_path} folder were previously stored"
                + f" in {file_name}. If you added new instances after the creation, then please "
                + "delete this file and call find function again. It will create it again."
           )

        with open(f"{db_path}/{file_name}", "rb") as f:
            representations = pickle.load(f)

        if not silent:
            print("There are ", len(representations), " representations found in ", file_name)


    df = pd.DataFrame(representations, columns=["identity", f"{model_name}_representation"])

    # img path might have more than once face
    target_objs = functions.extract_faces(img=img_path, target_size=target_size, detector_backend=detector_backend, grayscale=False, enforce_detection=enforce_detection, align=align)

    resp_obj = []

    for target_img, target_region, _ in target_objs:
        target_embedding_obj = DeepFace.represent(img_path=target_img, model_name=model_name, enforce_detection=enforce_detection, detector_backend="skip", align=align, normalization=normalization)

        target_representation = target_embedding_obj[0]["embedding"]

        result_df = df.copy()  # df will be filtered in each img
        result_df["source_x"] = target_region["x"]
        result_df["source_y"] = target_region["y"]
        result_df["source_w"] = target_region["w"]
        result_df["source_h"] = target_region["h"]

        distances = []
        for index, instance in df.iterrows():
            source_representation = instance[f"{model_name}_representation"]

            if distance_metric == "cosine":
                distance = dst.findCosineDistance(source_representation, target_representation)
            elif distance_metric == "euclidean":
                distance = dst.findEuclideanDistance(source_representation, target_representation)
            elif distance_metric == "euclidean_l2":
                distance = dst.findEuclideanDistance(dst.l2_normalize(source_representation), dst.l2_normalize(target_representation))
            else:
                raise ValueError(f"invalid distance metric passes - {distance_metric}")

            distances.append(distance)

            # ---------------------------

        result_df[f"{model_name}_{distance_metric}"] = distances

        threshold = dst.findThreshold(model_name, distance_metric)
        result_df = result_df.drop(columns=[f"{model_name}_representation"])
        result_df = result_df[result_df[f"{model_name}_{distance_metric}"] <= threshold]
        result_df = result_df.sort_values(by=[f"{model_name}_{distance_metric}"], ascending=True).reset_index(drop=True)

        resp_obj.append(result_df)

    # -----------------------------------

    toc = time.time()

    if not silent:
        print("find function lasts ", toc - tic, " seconds")

    return resp_obj


## Classes

### Class for Video Management

In [9]:

class VideoManager:
    @staticmethod
    def extract_faces_from_frame(frame, counter):
        try:
            filename1 = f"{Config.FRAME_FOLDER}/Image {counter}.jpg"
            print(filename1)
            cv2.imwrite(filename1, frame)

            faces = RetinaFace.extract_faces(filename1, threshold=0.99, model=None, allow_upscaling=True)

            if not faces:
                print(f"No faces extracted in Frame {counter}.")
                return None

            for idx, face in enumerate(faces):
                idxx = idx + 1
                filename2 = f"Frame.{counter}.Face.{idxx}.jpg"
                print(filename2)
                result = os.path.join(Config.FACE_FOLDER, filename2)
                cv2.imwrite(result, face)

            return counter

        except Exception as e:
            print(f"Error processing Frame {counter}: {str(e)}")
            return None

    @staticmethod
    def process_video_to_faces(video_path):
        video = cv2.VideoCapture(video_path)

        if not os.path.exists(Config.FRAME_FOLDER):
            os.makedirs(Config.FRAME_FOLDER)

        fps = video.get(cv2.CAP_PROP_FPS)
        frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        seconds = round(frames / fps)

        sec = 0.0
        counter = 0
        global_counter = 0
        interval = Config.INTERVAL

        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = []

            while sec < seconds:
                t_msec = int(1000 * sec)
                video.set(cv2.CAP_PROP_POS_MSEC, t_msec)
                ret, frame = video.read()

                if ret:
                    futures.append(executor.submit(VideoManager.extract_faces_from_frame, frame.copy(), counter))
                    sec += interval
                    counter += 1
                    global_counter += 1
                else:
                    sec += interval
                    global_counter += 1

            concurrent.futures.wait(futures)

        video.release()
        cv2.destroyAllWindows()


### Class For Face Recognition

In [10]:
class FaceRecog:

    global idds
    idds = []

    def get_id_data(self, img_path, bid):

        print(f"Studing image: {img_path}")

        finds = find(img_path, db_path=os.path.join(Config.DIRECTORY_FOLDER, Config.PKL_FOLDER), model_name=Config.MODEL_NAME, distance_metric=Config.DISTANCE_METRIC, detector_backend=Config.DETECTOR_BACKEND, enforce_detection=False, align=True, normalization="base", silent=True, bid = bid)

        ids = (img_path, finds[0]['identity'][0].split('/')[1].split('.')[1])

        self.sort_ids(ids)

        return finds[0]['identity'][0].split('/')[1].split('.')[1]

    @staticmethod
    def sort_ids(ids):
        global idds
        idds.append(ids)


    def get_ids_from_faces(self, face_dir, bid):
        ids = []
        faces = [os.path.join(face_dir, a) for a in os.listdir(face_dir)]

        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = []

            for face in faces:
                future = executor.submit(FaceRecog.get_id_data, self, face, bid)
                futures.append(future)

            for future in concurrent.futures.as_completed(futures):
                try:
                    id = future.result()
                    ids.append(id)
                    print(id)
                except Exception as e:
                    print(f"An error occurred: {str(e)}")

        global idds

        return ids, idds

### Class For Cleaning

In [11]:
class Cleanup:
    @staticmethod
    def clean():

        trash = (os.path.join(Config.DIRECTORY_FOLDER,a) for a in [Config.FRAME_FOLDER, Config.FACE_FOLDER]) # Config.UPLOADS_FOLDER])


        try:
            for directory_path in trash:
                shutil.rmtree(directory_path)
                print(f'Directory "{directory_path}" and its contents have been successfully deleted.')
        except Exception as e:
            print(f'An error occurred: {e}')

## API

### Upload Form

In [12]:
@app.get("/", response_class=HTMLResponse)
async def get_upload_form(request: Request):
    html_content = """
    <html>
    <head>
        <title>File Upload</title>
    </head>
    <body>
        <h1>File Upload</h1>
        <form action="/upload/" method="post" enctype="multipart/form-data">
            <input type="file" name="file">
            <input type="submit" value="Upload">
        </form>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content, status_code=200)

### Uploading File

In [13]:
@app.post("/upload/")
async def upload_file(file: UploadFile):
    try:
        if file.filename is None:
            return JSONResponse(content={"error": "File has no name."}, status_code=400)

        file_extension = os.path.splitext(file.filename)[1]

        if file_extension.lower() != '.mp4':
            return JSONResponse(content={"error": "Only MP4 files are allowed."}, status_code=400)

        if not os.path.exists(Config.UPLOADS_FOLDER):
            os.makedirs(Config.UPLOADS_FOLDER)

        with open(f"{Config.UPLOADS_FOLDER}/{Config.VIDEO_NAME}", "wb") as f:
            f.write(file.file.read())

        return RedirectResponse(url="/postupload")

    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

### Upload Done

In [14]:
@app.post("/postupload", response_class=HTMLResponse)
async def postupload():
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Prerecording</title>
    </head>
    <body>
        <div class="center-text">
            <h1>Upload done boi!</h1>
        </div>

        <form method="GET" action="/process_video">
            <label for="bid">Enter your Buisness ID:</label>
            <input type="text" id="bid" name="bid" required>
            <button type="submit">Let's start processing!</button>
        </form>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)

### Video Processing

In [15]:
@app.get("/process_video", response_class=JSONResponse)
async def video_processing(request: Request, bid: str):
    fac = VideoManager()
    idd = FaceRecog()

    fac.process_video_to_faces(os.path.join(Config.UPLOADS_FOLDER, Config.VIDEO_NAME))

    ids, idds = idd.get_ids_from_faces(Config.FACE_FOLDER, bid)

    # Create an empty dictionary to store the counts
    count_dict = {}

    # Loop through the list and update the counts in the dictionary
    for num in ids:
        if num in count_dict:
            count_dict[num] += 1
        else:
            count_dict[num] = 1

    # Create a dictionary to return as the JSON response
    response_dict = {
        "ids": ids,
        "idds": idds,
        "counts": count_dict,
    }

    print("Cleaning now!!")

    print(response_dict)

    # Cleanup.clean()

    return JSONResponse(content=response_dict)

# Initiation

In [ ]:
if __name__ == "__main__":
    cc.run_app(app=app)

INFO:     Started server process [256]
INFO:uvicorn.error:Started server process [256]
INFO:     Waiting for application startup.


Public URL: NgrokTunnel: "https://f151-35-194-170-45.ngrok-free.app" -> "http://localhost:12000"


INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     152.58.16.250:0 - "GET / HTTP/1.1" 200 OK
INFO:     152.58.16.250:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     152.58.16.250:0 - "POST /upload/ HTTP/1.1" 307 Temporary Redirect
INFO:     152.58.16.250:0 - "POST /postupload HTTP/1.1" 200 OK
Video_Frames/Image 0.jpg
Video_Frames/Image 1.jpg
Video_Frames/Image 2.jpg
Video_Frames/Image 3.jpg
Video_Frames/Image 4.jpg
Video_Frames/Image 5.jpg
retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5

retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
retinaface.h5 will be downloaded from the url https://github.com/serengil/deepfac

Downloading...Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5

From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5

  0%|          | 0.00/119M [00:00<?, ?B/s]Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5


  0%|          | 0.00/119M [00:00<?, ?B/s]
  0%|          | 524k/119M [00:00<00:52, 2.25MB/s]Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5



  0%|          | 0.00/119M [00:00<?, ?B/s]
  3%|▎         | 3.67M/119M [00:00<00:08, 13.3MB/s]

  0%|          | 524k/119M [00:00<00:59, 2.00MB/s]Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaf

Frame.0.Face.1.jpg
Frame.0.Face.2.jpg
Video_Frames/Image 6.jpg
Frame.1.Face.1.jpg
Video_Frames/Image 7.jpg
Frame.3.Face.1.jpg
Video_Frames/Image 8.jpg
Frame.5.Face.1.jpg
Video_Frames/Image 9.jpg
Frame.7.Face.1.jpg
Video_Frames/Image 10.jpg
Frame.2.Face.1.jpg
Video_Frames/Image 11.jpg
Frame.9.Face.1.jpg
Frame.9.Face.2.jpg
Video_Frames/Image 12.jpg
Frame.4.Face.1.jpg
Video_Frames/Image 13.jpg
Frame.11.Face.1.jpg
Frame.11.Face.2.jpg
Video_Frames/Image 14.jpg
Frame.6.Face.1.jpg
Video_Frames/Image 15.jpg
Frame.13.Face.1.jpg
Frame.13.Face.2.jpg
Video_Frames/Image 16.jpg
Frame.8.Face.1.jpg
Video_Frames/Image 17.jpg
Frame.15.Face.1.jpg
Frame.15.Face.2.jpg
Video_Frames/Image 18.jpg
Frame.10.Face.1.jpg
Frame.10.Face.2.jpg
Video_Frames/Image 19.jpg
Frame.17.Face.1.jpg
Frame.17.Face.2.jpg
Video_Frames/Image 20.jpg
Frame.12.Face.1.jpg
Frame.12.Face.2.jpg
Video_Frames/Image 21.jpg
Frame.19.Face.1.jpg
Frame.19.Face.2.jpg
Video_Frames/Image 22.jpg
Frame.14.Face.1.jpg
Frame.14.Face.2.jpg
Video_Frames/I